<a href="https://colab.research.google.com/github/jebs-hub/MicroDadosENEM/blob/main/code-pt/MicroDadosEnemAn%C3%A1lise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visão geral

Com o objetivo de aprender a manipular dados com a biblioteca `Pandas`, foi feita uma análise dos dados do ENEM para responder duas perguntas principais:
- Porcentagem de acertos por questão e o quanto isso reflete no nível de dificuldade prévio de cada questão
- Quantidade de acertos por nota, para que seja possível estabelecer uma relação entre essas grandezas e observar valores atípicos.

## Entendendo o ENEM e o banco
ENEM é sigla para EXAME Nacional do Ensino Médio.
Como o próprio nome já diz, consiste em uma prova realizada todos os anos pelo governo brasileiro para avaliar o conhecimento de alunos que estão se formando no Ensino Médio. 
A prova é composta por 4 áreas do conhecimento:
- Ciências Humanas e suas tecnologias
- Ciências da Natureza e suas tecnologias
- Linguagens, Códigos e suas tecnologias
- Matemática e suas tecnologias
- Redação

Com exceção de Redação, até a edição de 2022, as provas foram de múltipla escolha. A nota é atribuída com base no sistema de [TRI](https://gauchazh.clicrbs.com.br/educacao-e-emprego/noticia/2021/11/como-funciona-a-tri-na-avaliacao-do-enem-ckw3zwd9c004a016fuvd3uu6k.html). Assim duas pessoas com o mesmo número de acertos não necessariamente tiram a mesma nota. De acordo com o funcionamento da TRI, acertar questões mais fáceis somam mais pontos para nota final, do que acertar questões difíceis. 

## O problema
Com base nisso, questões muito relevantes emergem entre estudantes:
- Quais as questões mais fáceis, e portanto, as que devo acertar mais?
- Qual a relação média de acertos x notas?
- Quantas questões preciso acertar para que minha nota fique entre X e Y?

A única base da dados a qual temos acesso são os [Microdados do ENEM](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem). O documento contém informações sobre os editais, os métodos aplicados para avaliação da TRI e informações de inscritos, tais como informações socioeconômicas, o código e cor da prova realizada por cada inscrito, bem como seu gabarito e a nota correspondente (ambas em uma string. Ex: "ABBBDE.."). 
Dada a dimensão nacional da prova, os dados são muito extensos, chegando a 5 milhões de linhas.
Além disso, existe um arquivo menor com dados referentes às questões, que informam o código da questão, em quais provas ela aparece e em que ordem ela aparece, seu nível de dificuldade (usado pelo TRI para atribuir nota),entre outros parâmetros.
Todavia, não há respostas diretas para as perguntas mencionadas acima. 
Esse nootebook contém um código justamente para fazer essa análise e trazer respostas. 

## Funcionamento do código
As operações principais realizadas por esse código são:
1) Filtrar somente estudantes que fizeram as 5 provas com coerência, isto é, não zeraram, faltaram ou anularam nenhuma prova. 
2) Para cada estudante, comparar as strings de gabarito e de resposta para contabilizar a quantidade de acertos e contabilizar a quantidade de acertos de cada questão.
3) Ordenar questões com base na porcentagem para listas as mais fáceis e mais difíceis.
4) Construir histogramas e gráficos para analisar a relação de acertos e notas, com isso, buscar acertos mínimos e máximos para uma nota alvo, ou fazendo o processo contrário, buscar notas máximas e mínimas para um número de acertos alvo. 
5) Por fim, análises secundárias para o propósito desse projeto podem ser feitas: relação entre nota e nível socioenconômico, ano de formação no ensino médio, entre outros parâmetros.

A estrutura do que deve ser feito em si é simples. A maior dificuldade encontrada foi como organizar e padronizar todos os dados. Uma mesma provas tem várias versões, com questões em ordens diferentes. Logo, para duas respostas de dois alunos, "AABBB.." e "ACDDEA..", não significa que as questões na posição 0 da string sejam as mesmas, as provas podem ter cores diferentes e isso deve ser checado a cada contagem. 
Além disso, há casos em que, por motivos de desastres e demais problemas no dia da aplicação, a prova é aplicada em outro dia, então há duas provas com conjunto de questões totalmente diferentes e sem intersecção nenhuma. Outros detalhes, mais específicos foram surgindo e tiveram que ser tratados, como a opção pela língua espanhol ou inglês nas provas, que afeta o formato da string de resposta. 

## Saídas

Publiquei [planilhas](https://drive.google.com/drive/folders/1yVT-PJZ7oHBNL1bGHtw22vdQh4r-DzYS?usp=sharing) com gráficos e resultados mais interessantes do ponto de vista das pessoas que estão se preparando para a prova. O acesso a elas é livre.

Publiquei um [vídeo](https://www.youtube.com/watch?v=gUJe4yOGFLM) mostrando as análises. Não é um tutorial para o desenvolvimento do código! Mas explica o que foi feito, como funcionam os dados, mostra os dados finais sendo explorados e mostra como usar a planilha criada.




# Configuração inicial
Importando bibliotecas, lendo dados e ordenando vetores.

In [ ]:
import pandas as pd                    #importando bibliotecas importantes
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#importando colunas com dados úteis e incluindo dó quem tem nota válida para todas as áreas
fields = ['TP_ANO_CONCLUIU','TP_LINGUA', 'NU_NOTA_MT', 'TX_RESPOSTAS_MT','NU_NOTA_CN', 'TX_RESPOSTAS_CN','NU_NOTA_CH', 'TX_RESPOSTAS_CH', 'NU_NOTA_LC', 'TX_RESPOSTAS_LC', 'CO_PROVA_LC','CO_PROVA_MT','CO_PROVA_CH','CO_PROVA_CN', 'TX_GABARITO_MT', 'TX_GABARITO_LC', 'TX_GABARITO_CN', 'TX_GABARITO_CH','SG_UF_PROVA']
dados = pd.read_csv('/content/drive/MyDrive/dados/2019/MICRODADOS_ENEM_2019.csv', sep=';', usecols=fields, encoding='latin-1', skip_blank_lines=True).dropna()

In [ ]:
#importar relações entre questões, provas, ID's
infos_questoes = pd.read_csv('/content/drive/MyDrive/dados/ITENS_PROVA_2019.csv', sep=';', encoding='latin-1', skip_blank_lines=True)

In [ ]:
infos_questoes.sort_values(['CO_ITEM','CO_PROVA'],inplace=True) #ordenando questões por item

# Analisando questões
Criando estrutura para armazenar a quantidade acertos de cada questão, que será contabilizada depois.
A estrutura é formada por vários `dicionários` , dois para cada área do conhecimento: um para número de acertos e outro para número de respondidas. As chaves são o código da prova e os valores, um vetor com 45 posições inicializadas com valor zero. A cada acerto/resposta incrementamos um no dicionário, chave e posição correspondentes.

In [ ]:
CO_ITENS = infos_questoes['CO_ITEM'].drop_duplicates()
ACERTOS_QUESTOES = pd.DataFrame(0,index=CO_ITENS,columns=['NU_ACERTOS','NU_RESPONDIDAS','PORCENTAGEM']) #craindo matriz para receber os acertos

In [ ]:
CO_PROVAS = infos_questoes[['CO_PROVA','SG_AREA']].drop_duplicates(subset='CO_PROVA')  #criando dicionários para salvar os códigos das
NU_RESPONDIDAS_PROVAS_LC = {}                                                          #questões e quantos acertos ela teve
NU_RESPONDIDAS_PROVAS_CH = {}
NU_RESPONDIDAS_PROVAS_CN = {}
NU_RESPONDIDAS_PROVAS_MT = {}
NU_ACERTOS_PROVAS_LC = {}
NU_ACERTOS_PROVAS_CH = {}
NU_ACERTOS_PROVAS_CN = {}
NU_ACERTOS_PROVAS_MT = {}
from pandas.io.parsers.c_parser_wrapper import is_index_col
for index, row in CO_PROVAS.iterrows():   #para cada pessoa que fez o Enem
  if(row['SG_AREA'] == 'LC'):
    NU_RESPONDIDAS_PROVAS_LC[row['CO_PROVA']] = [0]*50
    NU_ACERTOS_PROVAS_LC[row['CO_PROVA']] = [0]*50
  elif(row['SG_AREA'] == 'CH'):
    NU_RESPONDIDAS_PROVAS_CH[row['CO_PROVA']] = [0]*45
    NU_ACERTOS_PROVAS_CH[row['CO_PROVA']] = [0]*45
  elif(row['SG_AREA'] == 'CN'):
    NU_RESPONDIDAS_PROVAS_CN[row['CO_PROVA']] = [0]*45
    NU_ACERTOS_PROVAS_CN[row['CO_PROVA']] = [0]*45
  elif(row['SG_AREA'] == 'MT'):
    NU_RESPONDIDAS_PROVAS_MT[row['CO_PROVA']] = [0]*45
    NU_ACERTOS_PROVAS_MT[row['CO_PROVA']] = [0]*45

# Contando questões - o principal
Criando um data frame para armazenar a quantidade de acertos de cada estudante por questão e por prova. 
Iteramos sobre os dados, contando e salvando valores: quantidade de acertos do estudante e quantidade de acertos da questão, 
Assim, usamos também a estrutura de questões criada anteriormente.
Essa é a parte principal do código, o centro do que precisa ser feito.

In [ ]:
ALUNOS_ACERTOS_NOTAS = pd.DataFrame(0,index=dados.index, columns=['NU_ACERTOS_LC','NU_ACERTOS_CH','NU_ACERTOS_CN','NU_ACERTOS_MT']) 
#criando colunas para salvar os acertos dos alunos

In [ ]:
from pandas.io.parsers.c_parser_wrapper import is_index_col                           #passando por todos os alunos e salvando os acertos
for index, row in dados.iterrows():   #para cada pessoa que fez o Enem
    
    CO_PROVA_CN = row['CO_PROVA_CN']  #obtem código de cada prova que ela fez
    CO_PROVA_MT = row['CO_PROVA_MT']
    CO_PROVA_CH = row['CO_PROVA_CH']
    CO_PROVA_LC = row['CO_PROVA_LC']
    
    NU_ACERTOS_LC = 0      #somador de quantidade de acertos
    NU_ACERTOS_CH = 0
    NU_ACERTOS_CN = 0
    NU_ACERTOS_MT = 0

    for i in range(0,45): #passa pelas 45 questões
    
      if(row['TP_LINGUA']==0 and i < 5):                          #se a língua é 1 e estamos nas primeiras 5 questoes
        if(row['TX_RESPOSTAS_LC'][i]==row['TX_GABARITO_LC'][i]):
          NU_ACERTOS_PROVAS_LC[int(CO_PROVA_LC)][i]+=1
          NU_ACERTOS_LC+=1
      else:
        if(row['TX_RESPOSTAS_LC'][i+5]==row['TX_GABARITO_LC'][i+5]):
          NU_ACERTOS_PROVAS_LC[int(CO_PROVA_LC)][i+5]+=1
          NU_ACERTOS_LC+=1

      if(row['TX_RESPOSTAS_CH'][i]==row['TX_GABARITO_CH'][i]):   #contabiliza acerto nas respostas em geral
        NU_ACERTOS_PROVAS_CH[int(CO_PROVA_CH)][i]+=1              #e contabiliza acerto para aluno
        NU_ACERTOS_CH+=1

      if(row['TX_RESPOSTAS_CN'][i]==row['TX_GABARITO_CN'][i]):
        NU_ACERTOS_PROVAS_CN[int(CO_PROVA_CN)][i]+=1
        NU_ACERTOS_CN+=1

      if(row['TX_RESPOSTAS_MT'][i]==row['TX_GABARITO_MT'][i]):
        NU_ACERTOS_PROVAS_MT[int(CO_PROVA_MT)][i]+=1
        NU_ACERTOS_MT+=1

      if(row['TP_LINGUA'] == 1 and i < 5):                     #independnete se acertou ou não contabiliza como respondida
        NU_RESPONDIDAS_PROVAS_LC[int(CO_PROVA_LC)][i+5]+=1       #essa relação é diferente para LC por causa das linguagens
      elif(row['TP_LINGUA'] == 0 and i < 5):
        NU_RESPONDIDAS_PROVAS_LC[int(CO_PROVA_LC)][i]+=1
      else:
        NU_RESPONDIDAS_PROVAS_LC[int(CO_PROVA_LC)][i+5]+=1

      NU_RESPONDIDAS_PROVAS_CH[int(CO_PROVA_CH)][i]+=1  #relação é simples para as outras
      NU_RESPONDIDAS_PROVAS_CN[int(CO_PROVA_CN)][i]+=1
      NU_RESPONDIDAS_PROVAS_MT[int(CO_PROVA_MT)][i]+=1
    
    ALUNOS_ACERTOS_NOTAS.loc[index]['NU_ACERTOS_LC']=NU_ACERTOS_LC  #fora do for de questões atualizamos quanto o aluno acertou
    ALUNOS_ACERTOS_NOTAS.loc[index]['NU_ACERTOS_CH']=NU_ACERTOS_CH
    ALUNOS_ACERTOS_NOTAS.loc[index]['NU_ACERTOS_CN']=NU_ACERTOS_CN
    ALUNOS_ACERTOS_NOTAS.loc[index]['NU_ACERTOS_MT']=NU_ACERTOS_MT

In [ ]:
ANALISE_NOTAS = pd.concat([dados[fields], ALUNOS_ACERTOS_NOTAS], axis=1) #concatenando resultados

# Concatenando resultados

Com base no código da prova e na sua variação de posição por provas, vamos achar as questões correspondetes entre provas e somar os acertos e as respondidas, concatenando todos os resultados.

In [ ]:
for CO_PROVA in NU_ACERTOS_PROVAS_LC:   #relacionando e concatenando as mesmas questões para provas diferentes
  for i in range(0,50):
    index = i + 1                 
    if(i > 4):
      index-=5                      
    if(i<5):
      CO_ITEM = infos_questoes.loc[(infos_questoes['CO_PROVA'] == int(CO_PROVA)) & (infos_questoes['CO_POSICAO'] == index) & (infos_questoes['TP_LINGUA']==0) , 'CO_ITEM'].iloc[0]
    elif(i<10):
      CO_ITEM = infos_questoes.loc[(infos_questoes['CO_PROVA'] == int(CO_PROVA)) & (infos_questoes['CO_POSICAO'] == index) & (infos_questoes['TP_LINGUA']==1) , 'CO_ITEM'].iloc[0]
    else:
      CO_ITEM = infos_questoes.loc[(infos_questoes['CO_PROVA'] == int(CO_PROVA)) & (infos_questoes['CO_POSICAO'] == index) , 'CO_ITEM'].iloc[0]
    ACERTOS_QUESTOES.loc[CO_ITEM]['NU_ACERTOS'] += NU_ACERTOS_PROVAS_LC[CO_PROVA][i]
    ACERTOS_QUESTOES.loc[CO_ITEM]['NU_RESPONDIDAS'] += NU_RESPONDIDAS_PROVAS_LC[CO_PROVA][i]

In [ ]:
for CO_PROVA in NU_ACERTOS_PROVAS_CH:
  for i in range(0,45):
    index = i + 46
    CO_ITEM = infos_questoes.loc[(infos_questoes['CO_PROVA'] == int(CO_PROVA)) & (infos_questoes['CO_POSICAO'] == index) , 'CO_ITEM'].iloc[0]
    ACERTOS_QUESTOES.loc[CO_ITEM]['NU_ACERTOS'] += NU_ACERTOS_PROVAS_CH[CO_PROVA][i]
    ACERTOS_QUESTOES.loc[CO_ITEM]['NU_RESPONDIDAS'] += NU_RESPONDIDAS_PROVAS_CH[CO_PROVA][i]

In [ ]:
for CO_PROVA in NU_ACERTOS_PROVAS_CN:
  for i in range(0,45):
    index = i + 91
    CO_ITEM = infos_questoes.loc[(infos_questoes['CO_PROVA'] == int(CO_PROVA)) & (infos_questoes['CO_POSICAO'] == index) , 'CO_ITEM'].iloc[0]
    ACERTOS_QUESTOES.loc[CO_ITEM]['NU_ACERTOS'] += NU_ACERTOS_PROVAS_CN[CO_PROVA][i]
    ACERTOS_QUESTOES.loc[CO_ITEM]['NU_RESPONDIDAS'] += NU_RESPONDIDAS_PROVAS_CN[CO_PROVA][i]

In [ ]:
for CO_PROVA in NU_ACERTOS_PROVAS_MT:
  for i in range(0,45):
    index = i + 136
    CO_ITEM = infos_questoes.loc[(infos_questoes['CO_PROVA'] == int(CO_PROVA)) & (infos_questoes['CO_POSICAO'] == index) , 'CO_ITEM'].iloc[0]
    ACERTOS_QUESTOES.loc[CO_ITEM]['NU_ACERTOS'] += NU_ACERTOS_PROVAS_MT[CO_PROVA][i]
    ACERTOS_QUESTOES.loc[CO_ITEM]['NU_RESPONDIDAS'] += NU_RESPONDIDAS_PROVAS_MT[CO_PROVA][i]

# A justando e escrevendo resultado

Após coletar e concatenar resultados, ajustamos e deixamos em um formato para salvar. 

In [ ]:
infos_questoes = infos_questoes.drop_duplicates(subset='CO_ITEM')  #ajustes finais

In [ ]:
ACERTOS_QUESTOES = ACERTOS_QUESTOES.reset_index()

In [ ]:
infos_questoes = infos_questoes.reset_index()

In [ ]:
ANALISE_QUESTOES = pd.concat([infos_questoes, ACERTOS_QUESTOES], axis=1)

In [ ]:
ANALISE_QUESTOES = ANALISE_QUESTOES.loc[:,~ANALISE_QUESTOES.columns.duplicated()]

In [ ]:
ANALISE_QUESTOES['PORCENTAGEM'] = ANALISE_QUESTOES['NU_ACERTOS']/ANALISE_QUESTOES['NU_RESPONDIDAS']*100

In [ ]:
ANALISE_QUESTOES.to_csv('/content/drive/MyDrive/dados/ANALISE_QUESTOES_2019.csv', index=False)  #salvando resultado
ANALISE_NOTAS.to_csv('/content/drive/MyDrive/dados/ANALISE_ENEM_2019.csv', index=False)

# Fazendo análises e gráficos
Esses são alguns exemplo das respostas que querememos buscar

## Variação da nota por número de acerto

In [ ]:
MAX = ANALISE_NOTAS[ANALISE_NOTAS['NU_ACERTOS_MT']==40]['NU_NOTA_MT'].max()

In [ ]:
MIN = ANALISE_NOTAS[ANALISE_NOTAS['NU_ACERTOS_MT']==40]['NU_NOTA_MT'].min()

In [ ]:
MIN = ANALISE_NOTAS[ANALISE_NOTAS['NU_NOTA_MT']>=800]['NU_ACERTOS_MT'].idxmin()

In [ ]:
ANALISE_NOTAS.loc[MIN]

In [ ]:
MAX - MIN

In [ ]:
ANALISE_NOTAS[(ANALISE_NOTAS['NU_ACERTOS_MT']==42)]

In [ ]:
#Histograma
acertos_MT = ANALISE_NOTAS[ANALISE_NOTAS['NU_ACERTOS_MT']==25]
notas = acertos_MT[['NU_NOTA_MT']].to_numpy()
plt.title('Frequência de notas por acerto')
plt.xlabel('Notas')
plt.ylabel('Frequência Absoluta')
plt.hist(notas, 10, rwidth=0.9)
plt.show()